In [1]:
#sparkify

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1708709776573_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# loading libraries
import pyspark
from pyspark import SparkConf
from pyspark import SparkFiles

from pyspark.sql import SparkSession

from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import TimestampType

from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, desc, countDistinct, first
from pyspark.sql.functions import udf, trim, expr
from pyspark.sql.functions import to_date, year, month, dayofmonth, dayofweek, hour, date_format, substring, datediff
from pyspark.sql.functions import sum as spark_sum, avg as spark_avg

from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, MinMaxScaler, StandardScaler
from pyspark.ml import Pipeline

from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier

from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Starting application

In [3]:
spark = SparkSession \
    .builder \
    .appName("Training models") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
path = "s3://sparkifylargedataset/user_data_12GB.json"  #prepared dataset to use for training
df = spark.read.json(path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df.take(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(About=1, Add Friend=5, Add to Playlist=4, Cancel=0, Cancellation Confirmation=0, Downgrade=0, Error=0, Help=5, Home=21, Login=0, Logout=11, NextSong=225, Register=0, Roll Advert=17, Save Settings=2, Settings=7, Submit Downgrade=0, Submit Registration=0, Submit Upgrade=0, Thumbs Down=1, Thumbs Up=4, Upgrade=1, avg_count_session_per_day=33.77777777777778, avg_count_session_per_month=152.0, avg_items=32.411184210526315, avg_items_per_day=1094.7777777777778, avg_items_per_month=4926.5, avg_session_length=242.7014788444445, avg_session_length_per_day=6067.536971111112, avg_session_length_per_month=27303.916370000006, churn_label=0, gender='M', state='NY', total_items=9853, total_session_length=54607.83274000001, unique_artist_count=203, unique_song_count=217, userId='1001862'), Row(About=0, Add Friend=2, Add to Playlist=2, Cancel=0, Cancellation Confirmation=0, Downgrade=0, Error=0, Help=2, Home=10, Login=0, Logout=3, NextSong=111, Register=0, Roll Advert=6, Save Settings=0, Settings=0

In [7]:
for column_name in df.columns:  #replaces space in column names into _
    new_column_name = column_name.replace(" ", "_")
    df = df.withColumnRenamed(column_name, new_column_name)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
row = df.count()  # extracting number of rows
col = len(df.columns)  # extracting number of columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
print(f'Dimension of the Dataframe is: {(row,col)}')
print(f'Number of Rows are: {row}')
print(f'Number of Columns are: {col}')

print(df.printSchema())  #printing our all column names

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Dimension of the Dataframe is: (22278, 38)
Number of Rows are: 22278
Number of Columns are: 38
root
 |-- About: long (nullable = true)
 |-- Add_Friend: long (nullable = true)
 |-- Add_to_Playlist: long (nullable = true)
 |-- Cancel: long (nullable = true)
 |-- Cancellation_Confirmation: long (nullable = true)
 |-- Downgrade: long (nullable = true)
 |-- Error: long (nullable = true)
 |-- Help: long (nullable = true)
 |-- Home: long (nullable = true)
 |-- Login: long (nullable = true)
 |-- Logout: long (nullable = true)
 |-- NextSong: long (nullable = true)
 |-- Register: long (nullable = true)
 |-- Roll_Advert: long (nullable = true)
 |-- Save_Settings: long (nullable = true)
 |-- Settings: long (nullable = true)
 |-- Submit_Downgrade: long (nullable = true)
 |-- Submit_Registration: long (nullable = true)
 |-- Submit_Upgrade: long (nullable = true)
 |-- Thumbs_Down: long (nullable = true)
 |-- Thumbs_Up: long (nullable = true)
 |-- Upgrade: long (nullable = true)
 |-- avg_count_session

In [10]:
Dict_Null = {col: df.filter(df[col].isNull()).count() for col in df.columns}
Dict_Null

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'About': 0, 'Add_Friend': 0, 'Add_to_Playlist': 0, 'Cancel': 0, 'Cancellation_Confirmation': 0, 'Downgrade': 0, 'Error': 0, 'Help': 0, 'Home': 0, 'Login': 0, 'Logout': 0, 'NextSong': 0, 'Register': 0, 'Roll_Advert': 0, 'Save_Settings': 0, 'Settings': 0, 'Submit_Downgrade': 0, 'Submit_Registration': 0, 'Submit_Upgrade': 0, 'Thumbs_Down': 0, 'Thumbs_Up': 0, 'Upgrade': 0, 'avg_count_session_per_day': 0, 'avg_count_session_per_month': 0, 'avg_items': 0, 'avg_items_per_day': 0, 'avg_items_per_month': 0, 'avg_session_length': 17, 'avg_session_length_per_day': 17, 'avg_session_length_per_month': 17, 'churn_label': 0, 'gender': 1, 'state': 1, 'total_items': 0, 'total_session_length': 17, 'unique_artist_count': 0, 'unique_song_count': 0, 'userId': 0}

In [11]:
df = df.dropna()
Dict_Null = {col: df.filter(df[col].isNull()).count() for col in df.columns}
Dict_Null

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'About': 0, 'Add_Friend': 0, 'Add_to_Playlist': 0, 'Cancel': 0, 'Cancellation_Confirmation': 0, 'Downgrade': 0, 'Error': 0, 'Help': 0, 'Home': 0, 'Login': 0, 'Logout': 0, 'NextSong': 0, 'Register': 0, 'Roll_Advert': 0, 'Save_Settings': 0, 'Settings': 0, 'Submit_Downgrade': 0, 'Submit_Registration': 0, 'Submit_Upgrade': 0, 'Thumbs_Down': 0, 'Thumbs_Up': 0, 'Upgrade': 0, 'avg_count_session_per_day': 0, 'avg_count_session_per_month': 0, 'avg_items': 0, 'avg_items_per_day': 0, 'avg_items_per_month': 0, 'avg_session_length': 0, 'avg_session_length_per_day': 0, 'avg_session_length_per_month': 0, 'churn_label': 0, 'gender': 0, 'state': 0, 'total_items': 0, 'total_session_length': 0, 'unique_artist_count': 0, 'unique_song_count': 0, 'userId': 0}

In [12]:
row = df.count()  # extracting number of rows
col = len(df.columns)  # extracting number of columns
print(f'Dimension of the Dataframe is: {(row,col)}')
print(f'Number of Rows are: {row}')
print(f'Number of Columns are: {col}')

print(df.printSchema())  #printing our all column names

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Dimension of the Dataframe is: (22261, 38)
Number of Rows are: 22261
Number of Columns are: 38
root
 |-- About: long (nullable = true)
 |-- Add_Friend: long (nullable = true)
 |-- Add_to_Playlist: long (nullable = true)
 |-- Cancel: long (nullable = true)
 |-- Cancellation_Confirmation: long (nullable = true)
 |-- Downgrade: long (nullable = true)
 |-- Error: long (nullable = true)
 |-- Help: long (nullable = true)
 |-- Home: long (nullable = true)
 |-- Login: long (nullable = true)
 |-- Logout: long (nullable = true)
 |-- NextSong: long (nullable = true)
 |-- Register: long (nullable = true)
 |-- Roll_Advert: long (nullable = true)
 |-- Save_Settings: long (nullable = true)
 |-- Settings: long (nullable = true)
 |-- Submit_Downgrade: long (nullable = true)
 |-- Submit_Registration: long (nullable = true)
 |-- Submit_Upgrade: long (nullable = true)
 |-- Thumbs_Down: long (nullable = true)
 |-- Thumbs_Up: long (nullable = true)
 |-- Upgrade: long (nullable = true)
 |-- avg_count_session

In [13]:
gender_indexer = StringIndexer(inputCol="gender", outputCol="gender_index")
state_indexer = StringIndexer(inputCol="state", outputCol="state_index")
indexed_data = gender_indexer.fit(df).transform(df)
indexed_data = state_indexer.fit(indexed_data).transform(indexed_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
indexed_data.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(About=1, Add_Friend=5, Add_to_Playlist=4, Cancel=0, Cancellation_Confirmation=0, Downgrade=0, Error=0, Help=5, Home=21, Login=0, Logout=11, NextSong=225, Register=0, Roll_Advert=17, Save_Settings=2, Settings=7, Submit_Downgrade=0, Submit_Registration=0, Submit_Upgrade=0, Thumbs_Down=1, Thumbs_Up=4, Upgrade=1, avg_count_session_per_day=33.77777777777778, avg_count_session_per_month=152.0, avg_items=32.411184210526315, avg_items_per_day=1094.7777777777778, avg_items_per_month=4926.5, avg_session_length=242.7014788444445, avg_session_length_per_day=6067.536971111112, avg_session_length_per_month=27303.916370000006, churn_label=0, gender='M', state='NY', total_items=9853, total_session_length=54607.83274000001, unique_artist_count=203, unique_song_count=217, userId='1001862', gender_index=0.0, state_index=1.0)]

In [15]:
print('creating train, validation and test sets')

train_df, remaining_df = indexed_data.randomSplit([0.7, 0.3], seed=42)
# The second split: 50% of the remaining data for validation, 50% for testing
validation_df, test_df = remaining_df.randomSplit([0.5, 0.5], seed=42)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

creating train, validation and test sets

In [16]:
def create_encoders():
    gender_encoder = OneHotEncoder(inputCol="gender_index",
                                   outputCol="gender_encoded")
    state_encoder = OneHotEncoder(inputCol="state_index",
                                  outputCol="state_encoded")
    return [gender_encoder, state_encoder]


#feature assembler function
def create_feature_assembler():
    cols_for_assembler = [
        'unique_artist_count', 'total_session_length', 'avg_session_length',
        'unique_song_count', 'total_items', 'avg_items',
        'avg_session_length_per_month', 'avg_items_per_month',
        'avg_count_session_per_month', 'avg_session_length_per_day',
        'avg_items_per_day', 'avg_count_session_per_day', 'About', 'Add_Friend',
        'Add_to_Playlist', 'Downgrade', 'Error', 'Help', 'Home', 'Logout',
        'NextSong', 'Roll_Advert', 'Save_Settings', 'Settings',
        'Submit_Downgrade', 'Submit_Upgrade', 'Thumbs_Down', 'Thumbs_Up',
        'Upgrade', 'state_encoded', 'gender_encoded'
    ]

    assembler = VectorAssembler(inputCols=cols_for_assembler,
                                outputCol="features")
    return assembler


#scaler function
def create_scaler():
    scaler = StandardScaler(inputCol='features',
                            outputCol='scaled_features',
                            withStd=True,
                            withMean=True)
    return scaler


#label indexer function
def create_label_indexer():
    label_indexer = StringIndexer(inputCol='churn_label', outputCol='label')
    return label_indexer

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# creating a pipeline including the preprocessing steps
encoders = create_encoders()
assembler = create_feature_assembler()
scaler = create_scaler()
label_indexer = create_label_indexer()
transform_pipeline = Pipeline(
    stages=[*encoders, assembler, scaler, label_indexer])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
#fitting transform pipeline
fitted_transform_pipeline = transform_pipeline.fit(train_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
#preprocessing the train and validation sets separately to avoid leak
train_transformed_data = fitted_transform_pipeline.transform(train_df)
validation_transformed_data = fitted_transform_pipeline.transform(validation_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
#function that trains and evaluate a model
def train_and_evaluate_model(model, train_data, validation_data):

    trained_model = model.fit(train_data)
    predictions = trained_model.transform(validation_data)

    evaluator = BinaryClassificationEvaluator()
    auc = evaluator.evaluate(predictions)

    evaluator_acc = MulticlassClassificationEvaluator(metricName="accuracy")
    accuracy = evaluator_acc.evaluate(predictions)

    return trained_model, auc, accuracy, predictions

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
#defining my models
rf_model = RandomForestClassifier(featuresCol='scaled_features',
                                  labelCol='label')
lr_model = LogisticRegression(featuresCol='scaled_features', labelCol='label')
gbt_model = GBTClassifier(featuresCol='scaled_features', labelCol='label')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
#training and evaluating based on the function created above
rf_trained_model, rf_auc, rf_accuracy, rf_predictions = train_and_evaluate_model(
    rf_model, train_transformed_data, validation_transformed_data)
lr_trained_model, lr_auc, lr_accuracy, lr_predictions = train_and_evaluate_model(
    lr_model, train_transformed_data, validation_transformed_data)
gbt_trained_model, gbt_auc, gbt_accuracy, gbt_predictions = train_and_evaluate_model(
    gbt_model, train_transformed_data, validation_transformed_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
rf_predictions.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(About=0, Add_Friend=0, Add_to_Playlist=0, Cancel=0, Cancellation_Confirmation=0, Downgrade=0, Error=0, Help=0, Home=0, Login=0, Logout=1, NextSong=7, Register=0, Roll_Advert=0, Save_Settings=0, Settings=0, Submit_Downgrade=0, Submit_Registration=0, Submit_Upgrade=0, Thumbs_Down=0, Thumbs_Up=1, Upgrade=0, avg_count_session_per_day=9.0, avg_count_session_per_month=9.0, avg_items=4.0, avg_items_per_day=36.0, avg_items_per_month=36.0, avg_session_length=295.04514428571434, avg_session_length_per_day=2065.3160100000005, avg_session_length_per_month=2065.3160100000005, churn_label=0, gender='F', state='GA', total_items=36, total_session_length=2065.3160100000005, unique_artist_count=7, unique_song_count=7, userId='1151342', gender_index=1.0, state_index=7.0, gender_encoded=SparseVector(1, {}), state_encoded=SparseVector(50, {7: 1.0}), features=SparseVector(80, {0: 7.0, 1: 2065.316, 2: 295.0451, 3: 7.0, 4: 36.0, 5: 4.0, 6: 2065.316, 7: 36.0, 8: 9.0, 9: 2065.316, 10: 36.0, 11: 9.0, 19: 1.

In [24]:
# table for Accuracy, Precision, Recall, and F1 Score for each model
models = [rf_trained_model, lr_trained_model, gbt_trained_model]
model_names = ["Random Forest", "Logistic Regression", "Gradient Boosted Trees"]
metrics_list = []

# Loop through each model
for i, model in enumerate(models):
    # Make predictions on the test set
    predictions = model.transform(validation_transformed_data)

    # Instantiate MulticlassClassificationEvaluator
    evaluator_multiclass = MulticlassClassificationEvaluator()

    # Evaluate metrics
    f1_score = evaluator_multiclass.evaluate(
        predictions, {evaluator_multiclass.metricName: "f1"})

    accuracy = evaluator_multiclass.evaluate(
        predictions, {evaluator_multiclass.metricName: "accuracy"})
    precision = evaluator_multiclass.evaluate(
        predictions, {evaluator_multiclass.metricName: "weightedPrecision"})
    recall = evaluator_multiclass.evaluate(
        predictions, {evaluator_multiclass.metricName: "weightedRecall"})

    # adding area under ROC
    evaluator = BinaryClassificationEvaluator()
    auc = evaluator.evaluate(predictions)

    # appending metrics to list
    metrics_list.append(
        (f"Model {model}", f1_score, auc, accuracy, precision, recall))

# Create a DataFrame from the list of metrics
metrics_df = spark.createDataFrame(metrics_list, [
    "Model",
    "F1 Score",
    "Area under ROC",
    "Accuracy",
    "Precision",
    "Recall",
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
metrics_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+------------------+------------------+------------------+------------------+
|               Model|          F1 Score|    Area under ROC|          Accuracy|         Precision|            Recall|
+--------------------+------------------+------------------+------------------+------------------+------------------+
|Model RandomFores...|0.6735803457305138|0.7645098827504785|0.7703190848886213|0.7812909406933741|0.7703190848886213|
|Model LogisticReg...|0.8274469932401357|0.8234603366852586|0.8473810957254666|  0.84639093123161|0.8473810957254666|
|Model GBTClassifi...|0.8280685268710538|0.8270544111061352|0.8470800722456352|0.8446007338709312|0.8470800722456352|
+--------------------+------------------+------------------+------------------+------------------+------------------+

In [26]:
paramgrid = ParamGridBuilder()\
            .build()

evaluator = MulticlassClassificationEvaluator(metricName="f1")

crossval = CrossValidator(estimator=gbt_model,
                          estimatorParamMaps=paramgrid,
                          evaluator=evaluator,
                          numFolds=5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
cv_gbt_model = crossval.fit(train_transformed_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkStatementCancellationFailedException: Interrupted by user but Livy failed to cancel the Spark statement. The Livy session might have become unusable.

In [ ]:
# Evaluate best model

In [ ]:
predictions = best_model.transform(transformed_test_data)

# Evaluate the model using a BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC')
area_under_roc = evaluator.evaluate(predictions)

# Print the Area Under ROC
print("Area Under ROC on Test Set: {:.4f}".format(area_under_roc))

accuracy = evaluator_multiclass.evaluate(
    predictions, {evaluator_multiclass.metricName: "accuracy"})
precision = evaluator_multiclass.evaluate(
    predictions, {evaluator_multiclass.metricName: "weightedPrecision"})
recall = evaluator_multiclass.evaluate(
    predictions, {evaluator_multiclass.metricName: "weightedRecall"})

f1_score = evaluator_multiclass.evaluate(
    predictions, {evaluator_multiclass.metricName: "f1"})

print(f'F1 score for best LR model: {f1_score}')
print(f'accuracy score for best LR model: {accuracy}')
print(f'precision score for best LR model: {precision}')
print(f'recall score for best LR model: {recall}')

In [ ]:
#make predictions on the test set based on the best model
test_transformed_data = train_pipeline.transform(test_data)
test_predictions = best_model.transform(test_transformed_data)

test_predictions.show()

In [ ]:
# showing users predicted to churn
selected_columns = ["userId", "prediction", "probability", "label"]

selected_predictions = test_predictions.select(*selected_columns)

churn_predictions = selected_predictions.filter(
    selected_predictions["prediction"] == 1.0)

churn_predictions.show()

In [ ]:
# creating a confusion matrix for the test dataset

# Extract true positive, false positive, true negative, and false negative counts
tp = test_predictions.filter((test_predictions["prediction"] == 1.0) &
                             (test_predictions["label"] == 1.0)).count()
fp = test_predictions.filter((test_predictions["prediction"] == 1.0) &
                             (test_predictions["label"] == 0.0)).count()
tn = test_predictions.filter((test_predictions["prediction"] == 0.0) &
                             (test_predictions["label"] == 0.0)).count()
fn = test_predictions.filter((test_predictions["prediction"] == 0.0) &
                             (test_predictions["label"] == 1.0)).count()

# Display confusion matrix
conf_matrix = [[tn, fp], [fn, tp]]
print("Confusion Matrix for best model on test dataset:")
print(conf_matrix)

In [ ]:
test_predictions.write.parquet(
    path="s3://sparkifylargedataset/test_predictions", mode="overwrite")